In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Prepare the dataset (Assume a folder structure with subfolders for each class)
def load_images_from_folder(data_directory):
    images = []
    labels = []
    label_names = os.listdir(data_directory)  # Class names (subfolders)

    for label_name in label_names:
        label_dir = os.path.join(data_directory, label_name)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (64, 64))  # Resize image to 64x64
                    images.append(img)
                    labels.append(label_name)

    return np.array(images), np.array(labels)

# Step 2: Load images and labels
data_directory = '/path/to/your/data'  # Set the path to your dataset
images, labels = load_images_from_folder(data_directory)

# Step 3: Encode labels (Convert class labels to integers)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 4: Normalize the images (Scale the pixel values to [0, 1])
images = images / 255.0

# Step 5: One-hot encode the labels for multiclass classification
encoded_labels = to_categorical(encoded_labels)

# Step 6: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Step 7: Data Augmentation (optional, to improve model performance)
train_datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.2, horizontal_flip=True)
train_datagen.fit(X_train)

# Step 8: Build CNN model for multiclass classification
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the 2D matrix to 1D vector
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))

# Output layer (softmax for multiclass classification)
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Step 9: Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 10: Train the model
model.fit(train_datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Step 11: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Step 12: Make predictions on new images
test_image_path = '/path/to/test_image.jpg'  # Path to a test image
test_image = cv2.imread(test_image_path)
test_image = cv2.resize(test_image, (64, 64))
test_image = test_image / 255.0  # Normalize
test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

# Predict the class of the test image
predicted_class = model.predict(test_image)
predicted_class_label = label_encoder.inverse_transform([np.argmax(predicted_class)])
print(f"Predicted class: {predicted_class_label[0]}")
